In [12]:
import pandas as pd
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import DataLoader, TensorDataset

# Örnek veri yükleme
data = pd.read_csv("birlesik_data.csv")  # haberler.csv dosyanızı uygun şekilde yükleyin

# Veri temizleme fonksiyonu
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Fazla boşlukları kaldırma
    text = re.sub(r'\W', ' ', text)  # Alfasayısal olmayan karakterleri kaldırma
    text = text.lower()  # Küçük harfe çevirme
    return text

data['cleaned_text'] = data['content'].apply(clean_text)

# Örnek etiketli veri (manuel olarak oluşturun veya mevcut verilerden seçin)
labeled_data = pd.DataFrame({
    'text': [
        "Küresel ısınma dünya çapında ciddi etkiler yaratıyor...",
        "Ekonomi bakanı yeni teşvik paketini açıkladı...",
        "Deniz seviyelerinin yükselmesi kıyı şehirlerini tehdit ediyor...",
        "Yeni bir güneş enerjisi santrali açıldı...",
        "Yerel halk, orman yangınlarına karşı önlem alıyor...",
        "Teknoloji şirketi yeni bir akıllı telefon tanıttı...",
        "Plastik atıkların okyanuslara zarar verdiği tespit edildi...",
        "Uluslararası anlaşmalar iklim değişikliğiyle mücadeleyi hedefliyor...",
        "Spor müsabakalarında büyük heyecan yaşandı...",
        "Yeni tarım teknikleri sürdürülebilirlik sağlıyor...",
        "Hava kirliliği insan sağlığını olumsuz etkiliyor...",
        "Fosil yakıtların kullanımının azaltılması gerektiği belirtiliyor...",
        "Ormanlar karbon emilimi için kritik öneme sahip...",
        "Yeni yürüyüş parkuru doğaseverlerin ilgisini çekiyor...",
        "İklim değişikliği nedeniyle kuraklık artıyor...",
        "Elektrikli araçların yaygınlaşması karbon ayak izini azaltıyor...",
        "Yerel yönetimler geri dönüşüm programlarını artırıyor...",
        "Yapay zeka alanında büyük ilerlemeler kaydedildi...",
        "Sanat galerisi yeni sergi açtı...",
        "Geleceğin enerji kaynakları yenilenebilir enerji olacak...",
        "Şehirde toplu taşıma sistemleri genişletiliyor...",
        "Küresel biyolojik çeşitlilik tehlikede...",
        "Çevre dostu binalar enerji tasarrufu sağlıyor...",
        "Uzay araştırmalarında yeni keşifler yapıldı...",
        "Geri dönüşüm fabrikası yeni tesis açtı...",
        "Ulusal parklar biyolojik çeşitliliği korumada önemli rol oynuyor...",
        "Orman yangınları doğal yaşamı tehdit ediyor...",
        "Temiz enerji teknolojileri hızla gelişiyor...",
        "Gıda israfını azaltma kampanyaları başlatıldı...",
        "Su kaynaklarının korunması için yeni düzenlemeler yapıldı..."
    ],
    'label': [
        1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1
    ]
})

# Eğitim ve doğrulama setlerine bölme
train_texts, val_texts, train_labels, val_labels = train_test_split(
    labeled_data['text'], labeled_data['label'], test_size=0.2, random_state=42
)

# TF-IDF vektörizasyonu
vectorizer = TfidfVectorizer(max_features=5000)
train_vectors = vectorizer.fit_transform(train_texts)
val_vectors = vectorizer.transform(val_texts)

# Verileri Tensor'a dönüştürme
train_vectors = torch.tensor(train_vectors.toarray(), dtype=torch.float32)
val_vectors = torch.tensor(val_vectors.toarray(), dtype=torch.float32)
train_labels = torch.tensor(train_labels.values, dtype=torch.long)
val_labels = torch.tensor(val_labels.values, dtype=torch.long)

# DataLoader
train_dataset = TensorDataset(train_vectors, train_labels)
val_dataset = TensorDataset(val_vectors, val_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [13]:
import torch.nn as nn
import torch.optim as optim

# Modeli oluşturma
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

input_size = train_vectors.shape[1]
model = SimpleNN(input_size)

# Kayıp fonksiyonu ve optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Modeli eğitme
for epoch in range(10):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print(f'Epoch: {epoch}, Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')


Epoch: 1, Validation Loss: 0.1125, Accuracy: 66.67%
Epoch: 2, Validation Loss: 0.1105, Accuracy: 66.67%
Epoch: 3, Validation Loss: 0.1085, Accuracy: 66.67%
Epoch: 4, Validation Loss: 0.1066, Accuracy: 66.67%
Epoch: 5, Validation Loss: 0.1049, Accuracy: 66.67%
Epoch: 6, Validation Loss: 0.1032, Accuracy: 66.67%
Epoch: 7, Validation Loss: 0.1018, Accuracy: 66.67%
Epoch: 8, Validation Loss: 0.1007, Accuracy: 66.67%
Epoch: 9, Validation Loss: 0.0999, Accuracy: 66.67%
Epoch: 10, Validation Loss: 0.0997, Accuracy: 66.67%


In [15]:

unlabeled_texts = data['cleaned_text'].tolist()  
unlabeled_vectors = vectorizer.transform(unlabeled_texts).toarray()


model.eval()
unlabeled_vectors = torch.tensor(unlabeled_vectors, dtype=torch.float32)
with torch.no_grad():
    pseudo_outputs = model(unlabeled_vectors)
    pseudo_labels = pseudo_outputs.argmax(dim=1).numpy()


combined_vectors = torch.cat((train_vectors, torch.tensor(unlabeled_vectors, dtype=torch.float32)), dim=0)
combined_labels = torch.cat((train_labels, torch.tensor(pseudo_labels, dtype=torch.long)), dim=0)

combined_dataset = TensorDataset(combined_vectors, combined_labels)
combined_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)


for epoch in range(10):
    model.train()
    for data, target in combined_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print(f'Epoch: {epoch+1}, Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')


IndexError: too many indices for tensor of dimension 2